In [1]:
%matplotlib tk
import numpy as np
import matplotlib.pyplot as plt
import scipy.ndimage
import astropy.io.fits

In [2]:
base_path = '/Users/harshmathur/Documents/CourseworkRepo/Spectroscopy Data/'
file_format = 'Flat_Prom_20170509_085123070_FORWARD_{}.fits'
num_of_files = 18
dark_master_path = 'dark_master.fits'

In [3]:
filename_list = list()

for i in range(1, num_of_files+1):
    filename_list.append(
        file_format.format(i)
    )

fits_list = list()

for _filename in filename_list:
    fits_list.append(
        astropy.io.fits.getdata(base_path+_filename)
    )

In [4]:
averaged_flat = np.average(fits_list, axis=0)

In [6]:
dark_master_path = 'dark_master.fits'
dark_image = astropy.io.fits.getdata(base_path+dark_master_path)

In [7]:
dark_image

array([[880.5929 , 880.72565, 880.34515, ..., 328.37167, 329.8761 ,
        328.54868],
       [880.1416 , 880.16815, 879.72565, ..., 329.86725, 329.54868,
        329.57523],
       [879.61945, 880.72565, 880.70795, ..., 331.16815, 330.79645,
        332.38937],
       ...,
       [881.7699 , 881.84955, 881.81415, ..., 879.58405, 878.92035,
        878.95575],
       [881.6106 , 882.20355, 881.4071 , ..., 879.1416 , 878.8407 ,
        878.74335],
       [882.0177 , 882.04425, 881.5044 , ..., 879.53094, 879.2832 ,
        879.4602 ]], dtype=float32)

In [22]:
plt.imshow(dark_image, cmap='gray')

In [8]:
averaged_flat.shape

(1, 1024, 512)

In [24]:
dark_image.shape

(1024, 512)

In [9]:
averaged_flat = averaged_flat[0]

In [26]:
averaged_flat.shape

(1024, 512)

In [10]:
dark_corrected_averaged_flat = averaged_flat - dark_image

In [31]:
plt.imshow(fits_list[0][0], cmap='gray')


In [11]:
max_shift, extent, up_sampling = 8, 10, 10

h_pix = dark_corrected_averaged_flat.shape[1]

display = dark_corrected_averaged_flat.copy()

plt.imshow(display, cmap='gray')

point = list(map(int, plt.ginput(1)[0]))


In [34]:
point[0]

111

In [12]:
point

[56, 297]

In [13]:
ref_col, y_beg, y_end = point[0], point[1] - extent / 2, point[1] + extent / 2

slit_ref = display[int(y_beg):int(y_end), ref_col]

normalised_slit = (slit_ref - slit_ref.mean()) / slit_ref.std()  # Normalize the selected slit profile

weights = np.zeros(shape=h_pix)

shift_ver = np.zeros(shape=h_pix)

In [14]:
for j in np.arange(h_pix):
    _slit = display[int(y_beg) - max_shift:int(y_end) + max_shift, j]
    weights[j] = abs(_slit.mean())
    _slit_normalised = (_slit - _slit.mean()) / _slit.std()  # Normalize the selected slit profile
    # Correlate
    correlation = np.correlate(
        scipy.ndimage.zoom(
            _slit_normalised,
            up_sampling
        ),
        scipy.ndimage.zoom(
            normalised_slit,
            up_sampling
        ),
        mode='valid'
    )
    shift_ver[j] = np.argmax(correlation) 

In [39]:
shift_ver

array([114., 105., 102., 104., 146., 149., 144., 144., 139., 133.,  81.,
        82.,  80.,  79.,  80.,  80.,  80.,  80., 100.,  81.,  81.,  81.,
        80.,  82., 102., 102., 102., 103.,  82.,  82.,  81.,  81., 106.,
       105., 106., 104., 107., 105., 100.,  84., 103., 105., 105., 108.,
       107., 108., 101., 105., 106.,  80., 102., 145., 142., 142., 144.,
       137., 145., 145., 145., 147.,  80.,  79.,  80.,  79.,  81.,  80.,
        80.,  81.,  81., 100.,  83.,  80.,  79.,  79.,  78., 145.,  81.,
        81.,  81.,  80.,  79.,  79.,  78.,  79.,  80.,  81.,  83.,  81.,
        81.,  81.,  81., 144., 145., 107., 103., 100.,  84., 101.,  81.,
        80.,  79.,  51.,  80.,  80.,  82., 102., 102.,  81.,  80.,  79.,
        79.,  52.,  80.,  82.,  81.,  80.,  79.,  79.,  79.,  80., 143.,
       136.,  80.,  81.,  81.,  83.,  79.,  78.,  77., 143., 140., 128.,
       105.,  81.,  80.,  79.,  79.,  78.,  79.,  80.,  81.,  81.,  79.,
        79.,  78.,  79.,  80., 100., 103., 101.,  8

In [40]:
plt.plot(shift_ver)
plt.show()

In [15]:
shift_ver = shift_ver / up_sampling - max_shift  # Convert the index to pixel shift

xfit_xinc = np.argwhere(abs(shift_ver) < max_shift)

yfit_yinc = shift_ver[xfit_xinc]

line_fit = np.polyfit(xfit_xinc.ravel(), yfit_yinc.ravel(), 1, w=np.nan_to_num(weights)[xfit_xinc].ravel())  # Fit a line

shift_ver_fit = line_fit[0] * np.arange(h_pix) + line_fit[1]

shift_ver_apply = -shift_ver_fit

plt.plot(xfit_xinc, yfit_yinc, 'k-', shift_ver_fit, 'k-')

plt.show()

In [17]:
x_corrected = dark_corrected_averaged_flat.copy()

for i in np.arange(h_pix):  # Calculate the shift corrected array
    scipy.ndimage.shift(dark_corrected_averaged_flat[:, i], shift_ver_apply[i], x_corrected[:, i], mode='nearest')

plt.imshow(x_corrected, cmap='gray')

plt.show()

In [18]:
plt.subplot(121)
plt.imshow(dark_corrected_averaged_flat, cmap='gray')
plt.subplot(122)
plt.imshow(x_corrected, cmap='gray')
plt.show()

/usr/local/lib/python3.7/site-packages/matplotlib/figure.py:98: MatplotlibDeprecationWarning: 
Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  "Adding an axes using the same arguments as a previous axes "


In [21]:
max_shift, extent, up_sampling = 10, 20, 10

v_pix = dark_corrected_averaged_flat.shape[0]

display = x_corrected.copy()

plt.imshow(display, cmap='gray')

point = list(map(int, plt.ginput(1)[0]))

ref_row, x_beg, x_end = int(point[1]), int(point[0] - extent / 2), int(point[0] + extent / 2)

line_ref = np.mean(display[int(ref_row) - 10:int(ref_row) + 10, int(x_beg):int(x_end)], 0)  # Reference line profile

normalised_line = (line_ref - line_ref.mean())/line_ref.std()  # Normalize the reference line profile

weights = np.zeros(shape=v_pix)

shift_hor = np.zeros(shape=v_pix)

In [20]:
point

[111, 330]

In [23]:
for j in np.arange(v_pix):
    if 5 <= j < v_pix-5:
        _line = np.mean(display[j - 5:j + 5, x_beg - max_shift:x_end + max_shift], axis=0)
    else:
        _line = display[j, x_beg - max_shift:x_end + max_shift]
    weights[j] = abs(_line.mean())
    _line_normalised = (_line - _line.mean()) / _line.std()  # Normalize the selected line profile
    correlation = np.correlate(
        scipy.ndimage.zoom(
            _line_normalised,
            up_sampling
        ),
        scipy.ndimage.zoom(
            normalised_line,
            up_sampling
        ),
        mode='valid'
    )
    shift_hor[j] = np.argmax(correlation)  # Get the index with maximum correlation
shift_hor = shift_hor / up_sampling - max_shift
xfit_yinc = np.argwhere((abs(shift_hor) < max_shift))
yfit_yinc = shift_hor[xfit_yinc]
polynomial_fit = np.polyfit(xfit_yinc.ravel(), yfit_yinc.ravel(), 2, w=np.nan_to_num(weights)[xfit_yinc].ravel())  # Fit a line
shift_hor_fit = polynomial_fit[0] * np.arange(v_pix) ** 2 + polynomial_fit[1] * np.arange(v_pix) + np.arange(v_pix)[2]  # Use the equation to calculate shifts
shift_hor_apply = -shift_hor_fit

In [24]:
plt.plot(xfit_yinc, yfit_yinc, 'k-', shift_hor_fit, 'k-')
plt.show()

In [25]:
y_corrected = x_corrected.copy()

for i in np.arange(v_pix):
    scipy.ndimage.shift(
        x_corrected[i, :],
        shift_hor_apply[i],
        y_corrected[i, :],
        mode='nearest'
    )

plt.imshow(y_corrected, cmap='gray')

plt.show()

In [26]:
plt.subplot(121)
plt.imshow(dark_corrected_averaged_flat, cmap='gray')
plt.subplot(122)
plt.imshow(y_corrected, cmap='gray')
plt.show()

In [28]:
rows_1 = np.arange(100, 400)
rows_2 = np.arange(600, 900)

cropped_array = np.append(y_corrected[rows_1], y_corrected[rows_2], axis=0)

line_median = np.median(cropped_array, axis=0)

line_median_normalised = line_median / line_median.max()

line_filter = scipy.ndimage.gaussian_filter1d(line_median_normalised, 2)

flat_line_removed = np.divide(y_corrected, line_filter)

plt.imshow(flat_line_removed, cmap='gray')

plt.show()

In [27]:
plt.close()

In [32]:
hdu = astropy.io.fits.PrimaryHDU(flat_line_removed)
hdul = astropy.io.fits.HDUList([hdu])
hdul.writeto('my_flat_master.fits')

# sunpy.io.fits.write('my_flat_master.fits', data=flat_line_removed)


In [33]:
raw_filename = 'KTT_Prom_20170509_075115500_FORWARD_3.fits'
flat_master = 'my_flat_master.fits'

In [34]:
raw_data = astropy.io.fits.getdata(base_path+raw_filename)
#data, header = sunpy.io.fits.read(base_path+raw_filename)[0]
flat_data = flat_line_removed

In [35]:
raw_data.shape

(1, 512, 512)

In [36]:
raw_data = raw_data[0]

In [37]:
raw_data.shape

(512, 512)

In [39]:
flat_line_removed.shape

(1024, 512)

In [45]:
binned_flat = np.zeros(shape=(512, 512))
i = 0
j = 0
while i < 1024:
    binned_flat[j] = np.add(flat_data[i], flat_data[i+1])
    i +=2
    j+=1


In [44]:
flat_master

'my_flat_master.fits'

In [47]:
binned_flat.shape

(512, 512)

In [48]:
plt.imshow(binned_flat, cmap='gray')
plt.show()

In [49]:
raw_data.shape

(512, 512)

In [50]:
binned_dark = np.zeros(shape=(512, 512))
i = 0
j = 0
while i < 1024:
    binned_dark[j] = np.add(dark_image[i], dark_image[i+1])
    i +=2
    j+=1


In [52]:
raw_corrected = np.divide(raw_data-binned_dark, binned_flat)

In [53]:
plt.imshow(raw_corrected, cmap='gray')
plt.show()

In [54]:
plt.plot(raw_corrected[250])
plt.show()

In [58]:
flat_corrected = np.divide(fits_list[0][0]-dark_image, flat_line_removed)

In [60]:
plt.imshow(flat_corrected, cmap='gray')
plt.show()

In [61]:
plt.plot(flat_corrected[250])
plt.show()

In [62]:
plt.plot(raw_corrected[250])
plt.show()